In [61]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,687 kB]

In [62]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-02 00:52:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.30MB/s    in 0.2s    

2020-11-02 00:52:43 (5.30 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [63]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [64]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...| 2015-08-31|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...|       Gift Card|          5|    

In [65]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [66]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R27ZP1F1CD0C3Y|          5|            0|          0|   N|                Y|
| RJ7RSBCHUDNNE|          5|            0|          0|   N|                Y|
|R1HVYBSKLQJI5S|          5|            0|          0|   N|                Y|
|R2HAXF0IIYQBIR|          1|            0|          0|   N|                Y|
| RNYLPX611NB7Q|          5|            0|          0|   N|                Y|
|R3ALA9XXMBEDZR|          5|            0|          0|   N|                Y|
|R3R8PHAVJFTPDF|          5|            0|          0|   N|                Y|
|R18WWEK8OIXE30|          5|            0|          0|   N|                Y|
|R1EGUNQON2J277|          1|            0|          0|   N|                Y|
|R21Z4M4L98CPU2|          5|            0|          0|   N|     

In [73]:
votes_df = vine_df.filter(vine_df['total_votes'] >= 20)

In [74]:
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R36PQ9D8L2AAOH|          5|           30|         32|   N|                Y|
|R23GHBE86II0SK|          5|           18|         21|   N|                Y|
|R18KVPY0TO33BM|          5|          241|        248|   N|                Y|
|R2ZB78BDM8BRCF|          1|           99|        113|   N|                Y|
|R2JMJOWGM7V7NX|          4|           20|         20|   N|                N|
|R10ORTN0I3G5B9|          1|           19|         26|   N|                Y|
|R1D1VPJBTP1WG8|          1|           38|         51|   N|                Y|
|R3Q3ULUTULAQYH|          1|           22|         29|   N|                Y|
|R1SHEELXA1IXQP|          5|           45|         57|   N|                Y|
|R1IQJ99JOPJOE4|          5|           73|         94|   N|     

In [77]:
percentdf = votes_df.filter((votes_df["helpful_votes"]/votes_df["total_votes"])>= .5)

In [78]:
percentdf.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R36PQ9D8L2AAOH|          5|           30|         32|   N|                Y|
|R23GHBE86II0SK|          5|           18|         21|   N|                Y|
|R18KVPY0TO33BM|          5|          241|        248|   N|                Y|
|R2ZB78BDM8BRCF|          1|           99|        113|   N|                Y|
|R2JMJOWGM7V7NX|          4|           20|         20|   N|                N|
|R10ORTN0I3G5B9|          1|           19|         26|   N|                Y|
|R1D1VPJBTP1WG8|          1|           38|         51|   N|                Y|
|R3Q3ULUTULAQYH|          1|           22|         29|   N|                Y|
|R1SHEELXA1IXQP|          5|           45|         57|   N|                Y|
|R1IQJ99JOPJOE4|          5|           73|         94|   N|     

In [111]:
vineyes_df = percentdf.filter(percentdf["vine"]=='Y')

In [112]:
vineno_df = percentdf.filter(percentdf["vine"]=='N')

In [ ]:
total_yesreviews_df = vineyes_df.groupby("review_id").agg({'review_id':'count'})

In [ ]:
total_noreviews_df = vineno_df.groupby("review_id").agg({'review_id':'count'})

In [ ]:
star_yesdf = total_yesreviews_df.filter(total_yesreviews_df['star_rating']==5)

In [ ]:
star_nodf = totalnoreviews_df.filter(total_noreviews_df['star_rating']==5)

In [ ]:
total_star_yesreviews_df = star_yesdf.filter(star_yesdf(["count(review_id)"]/["star_rating"])*100)

In [ ]:
total_star_noreviews_df = star_nodf.filter(star_nodf(["count(review_id)"]/["star_rating"])*100)